In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import *
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import *
from scipy import stats
from sklearn.model_selection import cross_val_score
from category_encoders import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn import tree
import math
import pickle
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
import warnings
warnings.filterwarnings('ignore')


In [2]:
encoder = TargetEncoder()
weatheDf = pd.read_csv("weather.csv")
weatheDf["encodedlocation"] = encoder.fit_transform(weatheDf['location'], weatheDf['rain'])
weatheDf.time_stamp = pd.to_datetime(weatheDf.time_stamp,unit='s')


In [3]:
#weatheDf.month = weatheDf.time_stamp.dt.month
weatheDf['month'] = weatheDf.time_stamp.dt.month
weatheDf.time_stamp = weatheDf.time_stamp.dt.hour
weatheDf = weatheDf.rename(
    columns={
        'time_stamp': 'hour'
    }
)
weatheDf.loc[(weatheDf.hour >= 6) & (
    weatheDf.hour < 12), 'time_of_day'] = 'Morning'
weatheDf.loc[(weatheDf.hour >= 12) & (weatheDf.hour < 15),
             'time_of_day'] = 'Afternoon'
weatheDf.loc[(weatheDf.hour >= 15) & (
    weatheDf.hour < 18), 'time_of_day'] = 'Evening'
weatheDf.loc[(weatheDf.hour >= 18) | (
    weatheDf.hour < 6), 'time_of_day'] = 'Night'
weatheDf = weatheDf.drop(['hour'], axis=1)
#turn month into categorical variable season
weatheDf.loc[(weatheDf.month >= 1) & (weatheDf.month < 4), 'season'] = 'Spring'
weatheDf.loc[(weatheDf.month >= 4) & (weatheDf.month < 7), 'season'] = 'Summer'
weatheDf.loc[(weatheDf.month >= 7) & (weatheDf.month < 10), 'season'] = 'Fall'
weatheDf.loc[(weatheDf.month >= 10) & (
    weatheDf.month < 13), 'season'] = 'Winter'
weatheDf = weatheDf.drop(['month'], axis=1)


In [4]:
scaler = MinMaxScaler()
scaler.fit(weatheDf.drop(['rain', 'location', 'time_of_day','season'], axis=1))
#wweatheDf = pd.DataFrame(scaler.transform(weatheDf.drop('rain',axis=1)), columns= weatheDf.drop('rain',axis=1).columns)
weatheDf1 =weatheDf.copy()
weatheDf.dropna(axis=0,inplace=True)
X = weatheDf.drop(['rain', 'location', 'time_of_day', 'season'], axis=1)
y = weatheDf.rain
x_pred = weatheDf1.drop(['rain', 'location', 'time_of_day','season'],axis=1)

#### Train the model to predict the missing values of rain feature

In [5]:
model = linear_model.LinearRegression()
poly_features = PolynomialFeatures(degree=3)
X_train_poly = poly_features.fit_transform(X)
x_pred_poly = poly_features.fit_transform(x_pred)
model.fit(X_train_poly, y)
y_train_predicted = model.predict(x_pred_poly)



In [6]:
weatheDf1.rain = y_train_predicted


In [7]:
def split_stratified_into_train_val_test(df_input, stratify_colname='y',
                                         frac_train=0.8,frac_test=0.2,
                                         random_state=None):
    X = df_input # Contains all columns.
    y = df_input[[stratify_colname]] # Dataframe of just the column on which to stratify.

    # Split original dataframe into train and temp dataframes.
    df_train, df_temp, y_train, y_temp = train_test_split(X,
                                                          y,
                                                          test_size=(1.0 - frac_train),
                                                          random_state=random_state)

 

    assert len(df_input) == len(df_train) + len(df_temp)

    return df_train, df_temp

In [8]:
weatherData = weatheDf1.drop('encodedlocation', axis=1)
avgWeather = weatherData.groupby(["location"]).mean().reset_index(drop=False)
sourceWeather = avgWeather.rename(
    columns={
        'location': 'source',
        'rain': 'source_rain',
        'temp': 'source_temp',
        'clouds': 'source_clouds',
        'pressure': 'source_pressure',
        'humidity': 'source_humidity',
        'wind': 'source_wind'
    }
)

destinationWeather = avgWeather.rename(
    columns={
        'location': 'destination',
        'rain': 'destination_rain',
        'temp': 'destination_temp',
        'clouds': 'destination_clouds',
        'pressure': 'destination_pressure',
        'humidity': 'destination_humidity',
        'wind': 'destination_wind'

    }
)


In [9]:
def testSample(sampleData, model):
    sampleData.time_stamp = pd.to_datetime(
        sampleData.time_stamp, unit="ms")
    sampleData['month'] = sampleData.time_stamp.dt.month
    sampleData.time_stamp = sampleData.time_stamp.dt.hour
    sampleData = sampleData.rename(
    columns={
        'time_stamp': 'hour'
    }
    )
    sampleData = sampleData.drop(['month'], axis=1)
    sampleData.loc[(sampleData.hour >= 0) & (
    sampleData.hour < 6), 'time_of_day'] = 'Night'
    sampleData.loc[(sampleData.hour >= 6) & (
    sampleData.hour < 12), 'time_of_day'] = 'Morning'
    sampleData.loc[(sampleData.hour >= 12) & (
    sampleData.hour < 18), 'time_of_day'] = 'Afternoon'
    sampleData.loc[(sampleData.hour >= 18) & (
    sampleData.hour < 24), 'time_of_day'] = 'Evening'
    sampleData = sampleData.drop(['hour'], axis=1)

    sampleData = sampleData.merge(sourceWeather, on=['source'])\
    .merge(destinationWeather, on=["destination"])
    sampleData.drop(['product_id'], axis=1, inplace=True)
    sampleData.drop(['id'], axis=1, inplace=True)
    top_feature = pickle.load(open("top_feature.pkl", "rb"))
    sampleData = sampleData[top_feature]
    for col in sampleData.select_dtypes(include='O').columns:
        encoder = pickle.load(open(f'{col}.pkl', 'rb'))
        sampleData[col] = encoder.transform(sampleData[col])
    y_tru = sampleData['RideCategory']
    sampleData.drop(['RideCategory'], axis=1, inplace=True)
    scaler = pickle.load(open('scaler.pkl', 'rb'))
    scaler.fit(sampleData)
    sampleData = pd.DataFrame(scaler.transform(sampleData))
    clrf = pickle.load(open(model, 'rb'))
    sample_pred = clrf.predict(sampleData)
    print(f'{model} acc', metrics.accuracy_score(y_tru, sample_pred) * 100.0)
    #print sample data and predicted data
    print(sample_pred[1:10])
    print(y_tru[1:10])


In [10]:
testSample(pd.read_csv('taxi-test-samples.csv'),'svm.pkl')


svm.pkl acc 75.0
[0 0 0]
1    0
2    0
3    3
Name: RideCategory, dtype: int64


In [11]:
testSample(pd.read_csv('taxi-test-samples.csv'),'ada_boost.pkl')

ada_boost.pkl acc 50.0
[0 2 2]
1    0
2    0
3    3
Name: RideCategory, dtype: int64


In [12]:
testSample(pd.read_csv('taxi-test-samples.csv'),'random_forest.pkl')

random_forest.pkl acc 25.0
[2 0 0]
1    0
2    0
3    3
Name: RideCategory, dtype: int64


In [13]:
testSample(pd.read_csv('taxi-test-samples.csv'),'naive_bayes.pkl')

naive_bayes.pkl acc 50.0
[0 0 0]
1    0
2    0
3    3
Name: RideCategory, dtype: int64


In [14]:
testSample(pd.read_csv('taxi-test-samples.csv'),'logistic_regression.pkl')

logistic_regression.pkl acc 50.0
[0 0 0]
1    0
2    0
3    3
Name: RideCategory, dtype: int64


In [15]:
testSample(pd.read_csv('taxi-test-samples.csv'),'light_gbm.pkl')

light_gbm.pkl acc 25.0
[2 0 0]
1    0
2    0
3    3
Name: RideCategory, dtype: int64


In [16]:
testSample(pd.read_csv('taxi-test-samples.csv'),'knn.pkl')

knn.pkl acc 25.0
[2 0 0]
1    0
2    0
3    3
Name: RideCategory, dtype: int64


In [17]:
testSample(pd.read_csv('taxi-test-samples.csv'),'gradient_boosting.pkl')

gradient_boosting.pkl acc 25.0
[2 0 0]
1    0
2    0
3    3
Name: RideCategory, dtype: int64


In [18]:
testSample(pd.read_csv('taxi-test-samples.csv'),'decision_tree.pkl')

decision_tree.pkl acc 25.0
[2 0 0]
1    0
2    0
3    3
Name: RideCategory, dtype: int64
